# 26-元类编程

## 📚 用途说明

**学习目标**：
- 深入理解Python元类的概念和工作原理
- 掌握元类的创建、自定义和应用场景
- 能够运用元类实现高级的类定制和框架设计

**前置要求**：
- 已完成25-设计模式应用学习
- 熟练掌握面向对象编程的所有概念
- 深刻理解Python的类创建机制和特殊方法

**与LangChain关联**：
- 元类是LangChain框架架构的核心技术
- LangChain使用元类实现组件的自动注册和发现
- 元类模式影响LangChain的扩展性和配置管理
- 为后续学习LangChain的框架设计和自定义组件做准备

---

## 🔢 知识点覆盖

### 3.8 元类编程 [⭐⭐进阶]
**知识点说明**：元类是创建类的类，是Python中最强大的元编程工具。通过元类，我们可以在类创建时动态修改类的定义，实现框架级别的功能定制。

**学习要求**：
- 掌握元类的基本概念和工作原理
- 理解元类的创建方法和应用场景
- 能够运用元类解决复杂的框架设计问题

**案例要求**：
- 实现完整的元类示例
- 进行元类编程练习
- 应用元类解决实际问题
- 验证点：能独立设计和实现元类

In [ ]:
print("🏗️ 元类基础概念:")
print("=" * 50)

# 1. 理解元类的本质
print(f"📝 1. 理解元类的本质:")

# Python中一切皆对象
print(f"   Python中一切皆对象:")

class SimpleClass:
    """简单类"""
    
    def __init__(self, value):
        self.value = value
    
    def show(self):
        return f"Value: {self.value}"

# 类也是对象
print(f"\n     类的类型:")
print(f"       SimpleClass 的类型: {type(SimpleClass)}")
print(f"       SimpleClass 是 type 的实例: {isinstance(SimpleClass, type)}")

# 对象的类型
obj = SimpleClass(42)
print(f"\n     对象的类型:")
print(f"       obj 的类型: {type(obj)}")
print(f"       obj 是 SimpleClass 的实例: {isinstance(obj, SimpleClass)}")

# type的层次结构
print(f"\n     type的层次结构:")
print(f"       type 的类型: {type(type)}")
print(f"       type 是 type 的实例: {isinstance(type, type)}")
print(f"       object 的类型: {type(object)}")
print(f"       SimpleClass 的父类: {SimpleClass.__bases__}")
print(f"       type 的父类: {type.__bases__}")

# 2. 动态创建类
print(f"\n📝 2. 动态创建类:")

# 使用type动态创建类
print(f"   使用type动态创建类:")

# 传统方式定义类
class TraditionalClass:
    """传统方式定义的类"""
    
    def __init__(self, name):
        self.name = name
    
    def greet(self):
        return f"Hello, {self.name}!"

# 使用type动态创建等价的类
def dynamic_init(self, name):
    self.name = name

def dynamic_greet(self):
    return f"Hello, {self.name}!"

DynamicClass = type(
    'DynamicClass',  # 类名
    (object,),       # 基类元组
    {
        '__init__': dynamic_init,
        'greet': dynamic_greet,
        'class_description': '动态创建的类'
    }
)

# 测试两种方式创建的类
print(f"\n     测试传统方式创建的类:")
traditional_obj = TraditionalClass("张三")
print(f"       {traditional_obj.greet()}")

print(f"\n     测试动态创建的类:")
dynamic_obj = DynamicClass("李四")
print(f"       {dynamic_obj.greet()}")
print(f"       类描述: {DynamicClass.class_description}")

# 验证类的等价性
print(f"\n     验证类的等价性:")
print(f"       TraditionalClass 方法: {[method for method in dir(TraditionalClass) if not method.startswith('_')]}")
print(f"       DynamicClass 方法: {[method for method in dir(DynamicClass) if not method.startswith('_')]}")

# 3. 简单元类
print(f"\n📝 3. 简单元类:")

class SimpleMeta(type):
    """简单元类"""
    
    def __new__(mcs, name, bases, namespace):
        print(f"   创建类: {name}")
        print(f"   基类: {bases}")
        print(f"   命名空间: {list(namespace.keys())}")
        
        # 添加类属性
        namespace['created_by'] = 'SimpleMeta'
        namespace['creation_time'] = '2023-11-30'
        
        # 创建类
        cls = super().__new__(mcs, name, bases, namespace)
        
        print(f"   类创建完成: {cls}")
        return cls

class MetaClassExample(metaclass=SimpleMeta):
    """使用元类的示例类"""
    
    def __init__(self, value):
        self.value = value
    
    def show(self):
        return f"Value: {self.value}"

print(f"\n     测试简单元类:")
meta_obj = MetaClassExample(100)
print(f"\n       对象方法: {meta_obj.show()}")
print(f"       类属性: created_by = {MetaClassExample.created_by}")
print(f"       类属性: creation_time = {MetaClassExample.creation_time}")

# 4. 元类方法拦截
print(f"\n📝 4. 元类方法拦截:")

class MethodInterceptorMeta(type):
    """方法拦截元类"""
    
    def __new__(mcs, name, bases, namespace):
        # 拦截所有方法，添加日志功能
        for key, value in namespace.items():
            if callable(value) and not key.startswith('__'):
                namespace[key] = mcs._add_logging(value)
        
        return super().__new__(mcs, name, bases, namespace)
    
    @staticmethod
    def _add_logging(func):
        """为方法添加日志功能"""
        def wrapper(*args, **kwargs):
            print(f"   [日志] 调用方法: {func.__name__}")
            result = func(*args, **kwargs)
            print(f"   [日志] 方法 {func.__name__} 执行完成")
            return result
        
        # 保持原函数的元信息
        wrapper.__name__ = func.__name__
        wrapper.__doc__ = func.__doc__
        return wrapper

class LoggedClass(metaclass=MethodInterceptorMeta):
    """自动添加日志的类"""
    
    def process_data(self, data):
        """处理数据"""
        return f"处理后的数据: {data}"
    
    def calculate(self, x, y):
        """计算"""
        return x + y

print(f"\n     测试方法拦截元类:")
logged_obj = LoggedClass()

print(f"\n       调用 process_data 方法:")
result1 = logged_obj.process_data("测试数据")
print(f"         结果: {result1}")

print(f"\n       调用 calculate 方法:")
result2 = logged_obj.calculate(10, 20)
print(f"         结果: {result2}")

print(f"\n✅ 元类基础概念完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 理解元类的本质和工作原理")
print(f"   ✓ 掌握动态创建类的方法")
print(f"   ✓ 能创建简单的元类并实现方法拦截")

### 元类高级应用 [⭐⭐进阶]
**知识点说明**：元类的高级应用包括属性验证、方法注册、单例模式实现、ORM框架等，展示了元类在框架设计中的强大能力。

**学习要求**：
- 掌握元类的高级应用技巧
- 理解元类在框架设计中的作用
- 能够运用元类解决复杂的架构问题

**案例要求**：
- 实现完整的元类高级应用示例
- 进行元类框架设计练习
- 应用元类解决实际问题
- 验证点：能独立设计基于元类的框架组件

In [ ]:
print("🔧 元类高级应用:")
print("=" * 50)

# 1. 属性验证元类
print(f"📝 1. 属性验证元类:")

class ValidationMeta(type):
    """属性验证元类"""
    
    def __new__(mcs, name, bases, namespace):
        # 获取字段定义
        fields = namespace.get('_fields', {})
        
        # 创建验证方法
        def __init__(self, **kwargs):
            for field_name, field_type in fields.items():
                if field_name in kwargs:
                    value = kwargs[field_name]
                    if not isinstance(value, field_type):
                        raise TypeError(f"字段 {field_name} 应该是 {field_type.__name__} 类型，但得到 {type(value).__name__}")
                    setattr(self, field_name, value)
                else:
                    setattr(self, field_name, None)
        
        # 创建验证方法
        def validate(self):
            errors = []
            for field_name, field_type in fields.items():
                value = getattr(self, field_name)
                if value is not None and not isinstance(value, field_type):
                    errors.append(f"字段 {field_name} 类型错误")
            return errors
        
        # 创建字典表示方法
        def to_dict(self):
            return {field: getattr(self, field) for field in fields}
        
        # 添加方法到命名空间
        namespace['__init__'] = __init__
        namespace['validate'] = validate
        namespace['to_dict'] = to_dict
        
        return super().__new__(mcs, name, bases, namespace)

class User(metaclass=ValidationMeta):
    """用户模型"""
    _fields = {
        'name': str,
        'age': int,
        'email': str,
        'active': bool
    }

class Product(metaclass=ValidationMeta):
    """产品模型"""
    _fields = {
        'title': str,
        'price': float,
        'quantity': int,
        'available': bool
    }

# 测试属性验证元类
print(f"   属性验证元类测试:")

print(f"\n     创建有效用户:")
try:
    user = User(name="张三", age=25, email="zhangsan@example.com", active=True)
    print(f"       用户创建成功: {user.to_dict()}")
    print(f"       验证结果: {user.validate()}")
except Exception as e:
    print(f"       错误: {e}")

print(f"\n     创建无效用户:")
try:
    invalid_user = User(name="李四", age="三十", email="lisi@example.com", active=True)
    print(f"       用户创建: {invalid_user.to_dict()}")
except Exception as e:
    print(f"       错误: {e}")

print(f"\n     创建产品:")
product = Product(title="手机", price=2999.99, quantity=100, available=True)
print(f"       产品创建成功: {product.to_dict()}")

# 2. 单例元类
print(f"\n📝 2. 单例元类:")

class SingletonMeta(type):
    """单例元类"""
    
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            print(f"   创建 {cls.__name__} 的唯一实例")
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        else:
            print(f"   返回已存在的 {cls.__name__} 实例")
        return cls._instances[cls]
    
    def get_instance(cls):
        """获取当前实例"""
        return cls._instances.get(cls)
    
    def clear_instance(cls):
        """清除实例（用于测试）"""
        if cls in cls._instances:
            del cls._instances[cls]
            print(f"   清除 {cls.__name__} 实例")

class DatabaseConnection(metaclass=SingletonMeta):
    """数据库连接单例"""
    
    def __init__(self, connection_string=None):
        if connection_string:
            self.connection_string = connection_string
            print(f"   设置连接字符串: {connection_string}")
        else:
            self.connection_string = "default_connection"
    
    def connect(self):
        print(f"   连接到: {self.connection_string}")
        return f"连接到 {self.connection_string} 成功"

class Logger(metaclass=SingletonMeta):
    """日志单例"""
    
    def __init__(self):
        if not hasattr(self, 'logs'):
            self.logs = []
            print(f"   初始化日志系统")
    
    def log(self, message):
        self.logs.append(message)
        print(f"   记录日志: {message}")
    
    def get_logs(self):
        return self.logs.copy()

# 测试单例元类
print(f"\n   单例元类测试:")

print(f"\n     数据库连接单例测试:")
db1 = DatabaseConnection("localhost:5432/mydb")
db2 = DatabaseConnection("localhost:5432/otherdb")

print(f"\n     实例比较:")
print(f"       db1 is db2: {db1 is db2}")
print(f"       db1 连接字符串: {db1.connection_string}")
print(f"       db2 连接字符串: {db2.connection_string}")

print(f"\n     日志单例测试:")
logger1 = Logger()
logger2 = Logger()

logger1.log("系统启动")
logger2.log("用户登录")

print(f"\n     实例比较:")
print(f"       logger1 is logger2: {logger1 is logger2}")
print(f"       共享日志: {logger1.get_logs()}")

# 3. 注册元类
print(f"\n📝 3. 注册元类:")

class RegistryMeta(type):
    """注册元类"""
    
    def __init__(cls, name, bases, namespace):
        super().__init__(name, bases, namespace)
        
        # 创建注册表
        if not hasattr(cls, '_registry'):
            cls._registry = {}
        
        # 注册类
        if name != 'BasePlugin':  # 不注册基类
            cls._registry[name] = cls
            print(f"   注册插件: {name}")
    
    def get_plugins(cls):
        """获取所有注册的插件"""
        return cls._registry.copy()
    
    def get_plugin(cls, name):
        """获取指定插件"""
        return cls._registry.get(name)
    
    def register_plugin(cls, plugin_class):
        """手动注册插件"""
        cls._registry[plugin_class.__name__] = plugin_class
        print(f"   手动注册插件: {plugin_class.__name__}")

class BasePlugin(metaclass=RegistryMeta):
    """插件基类"""
    
    def __init__(self):
        self.name = self.__class__.__name__
    
    def execute(self, *args, **kwargs):
        raise NotImplementedError("子类必须实现 execute 方法")
    
    def get_info(self):
        return f"插件: {self.name}"

class EmailPlugin(BasePlugin):
    """邮件插件"""
    
    def execute(self, recipient, message):
        print(f"   发送邮件到 {recipient}: {message}")
        return f"邮件已发送到 {recipient}"

class SMSPlugin(BasePlugin):
    """短信插件"""
    
    def execute(self, phone, message):
        print(f"   发送短信到 {phone}: {message}")
        return f"短信已发送到 {phone}"

class PushPlugin(BasePlugin):
    """推送插件"""
    
    def execute(self, device, message):
        print(f"   推送消息到设备 {device}: {message}")
        return f"消息已推送到设备 {device}"

# 测试注册元类
print(f"\n   注册元类测试:")

print(f"\n     获取所有注册的插件:")
plugins = BasePlugin.get_plugins()
for plugin_name, plugin_class in plugins.items():
    print(f"       {plugin_name}: {plugin_class}")

print(f"\n     使用插件:")
email_plugin = BasePlugin.get_plugin('EmailPlugin')()
sms_plugin = BasePlugin.get_plugin('SMSPlugin')()
push_plugin = BasePlugin.get_plugin('PushPlugin')()

email_plugin.execute("user@example.com", "Hello World")
sms_plugin.execute("13800138000", "验证码: 123456")
push_plugin.execute("device123", "新消息通知")

# 4. ORM元类
print(f"\n📝 4. ORM元类:")

class ORMMeta(type):
    """ORM元类"""
    
    def __new__(mcs, name, bases, namespace):
        # 收集字段定义
        fields = {}
        for key, value in namespace.items():
            if isinstance(value, Field):
                fields[key] = value
                del namespace[key]  # 从命名空间中移除字段定义
        
        # 添加字段信息到类
        namespace['_fields'] = fields
        namespace['_table_name'] = name.lower()
        
        # 创建初始化方法
        def __init__(self, **kwargs):
            for field_name, field in fields.items():
                if field_name in kwargs:
                    value = kwargs[field_name]
                    field.validate(value)
                    setattr(self, field_name, value)
                else:
                    setattr(self, field_name, field.default)
        
        # 创建保存方法
        def save(self):
            print(f"   保存 {self._table_name} 到数据库:")
            for field_name, field in self._fields.items():
                value = getattr(self, field_name)
                print(f"     {field_name}: {value} ({field.field_type.__name__})")
            return f"{self._table_name} 保存成功"
        
        # 创建查询方法
        @classmethod
        def query(cls, **kwargs):
            print(f"   查询 {cls._table_name} 表:")
            for field_name, value in kwargs.items():
                print(f"     WHERE {field_name} = {value}")
            return f"查询到 {len(kwargs)} 个条件的记录"
        
        # 添加方法到命名空间
        namespace['__init__'] = __init__
        namespace['save'] = save
        namespace['query'] = query
        
        return super().__new__(mcs, name, bases, namespace)

class Field:
    """字段类"""
    
    def __init__(self, field_type, default=None, required=True):
        self.field_type = field_type
        self.default = default
        self.required = required
    
    def validate(self, value):
        if value is None and self.required:
            raise ValueError("字段不能为空")
        if value is not None and not isinstance(value, self.field_type):
            raise TypeError(f"字段类型应该是 {self.field_type.__name__}")

# 创建字段类型
def StringField(default=None, required=True):
    return Field(str, default, required)

def IntegerField(default=None, required=True):
    return Field(int, default, required)

def FloatField(default=None, required=True):
    return Field(float, default, required)

def BooleanField(default=False, required=True):
    return Field(bool, default, required)

class User(metaclass=ORMMeta):
    """用户ORM模型"""
    name = StringField(required=True)
    age = IntegerField(default=0)
    email = StringField(required=True)
    active = BooleanField(default=True)

class Product(metaclass=ORMMeta):
    """产品ORM模型"""
    title = StringField(required=True)
    price = FloatField(default=0.0)
    quantity = IntegerField(default=0)
    available = BooleanField(default=True)

# 测试ORM元类
print(f"\n   ORM元类测试:")

print(f"\n     创建用户:")
user = User(name="张三", age=25, email="zhangsan@example.com")
print(f"       用户创建成功: {user._table_name}")
print(f"       字段信息: {list(user._fields.keys())}")

print(f"\n     保存用户:")
user.save()

print(f"\n     查询用户:")
result = User.query(name="张三", active=True)
print(f"       {result}")

print(f"\n     创建产品:")
product = Product(title="手机", price=2999.99, quantity=100)
product.save()

print(f"\n✅ 元类高级应用完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握元类的高级应用技巧")
print(f"   ✓ 理解元类在框架设计中的作用")
print(f"   ✓ 能运用元类解决复杂的架构问题")

### 实际框架案例 [⭐⭐进阶]
**知识点说明**：通过实际框架案例展示元类在大型项目中的应用，包括依赖注入、API框架、配置管理等场景。

**学习要求**：
- 掌握元类在实际框架中的应用
- 理解元类驱动的架构设计模式
- 能够设计基于元类的框架组件

**案例要求**：
- 实现完整的框架级元类应用
- 进行框架设计练习
- 应用元类解决实际框架问题
- 验证点：能独立设计基于元类的框架架构

In [ ]:
print("🏗️ 实际框架案例:")
print("=" * 50)

# 1. 依赖注入框架
print(f"📝 1. 依赖注入框架:")

class DIMeta(type):
    """依赖注入元类"""
    
    def __new__(mcs, name, bases, namespace):
        # 收集依赖注入标记
        dependencies = {}
        for key, value in namespace.items():
            if hasattr(value, '_inject'):
                dependencies[key] = value._inject
        
        # 创建初始化方法
        original_init = namespace.get('__init__')
        
        def __init__(self, *args, **kwargs):
            # 注入依赖
            for attr_name, dependency_type in dependencies.items():
                if hasattr(self, '_container'):
                    dependency = self._container.get(dependency_type)
                    setattr(self, attr_name, dependency)
                else:
                    # 创建默认实例
                    setattr(self, attr_name, dependency_type())
            
            # 调用原始初始化方法
            if original_init:
                original_init(self, *args, **kwargs)
        
        # 添加依赖信息
        namespace['_dependencies'] = dependencies
        namespace['__init__'] = __init__
        
        return super().__new__(mcs, name, bases, namespace)

def inject(dependency_type):
    """依赖注入装饰器"""
    def decorator(func):
        func._inject = dependency_type
        return func
    return decorator

class DIContainer:
    """依赖注入容器"""
    
    def __init__(self):
        self._services = {}
        self._singletons = {}
    
    def register(self, service_type, implementation=None, singleton=False):
        """注册服务"""
        if implementation is None:
            implementation = service_type
        
        self._services[service_type] = {
            'implementation': implementation,
            'singleton': singleton
        }
        print(f"   注册服务: {service_type.__name__} -> {implementation.__name__}")
    
    def get(self, service_type):
        """获取服务实例"""
        if service_type not in self._services:
            # 自动注册
            self.register(service_type)
        
        service_info = self._services[service_type]
        
        if service_info['singleton']:
            if service_type not in self._singletons:
                instance = service_info['implementation']()
                instance._container = self
                self._singletons[service_type] = instance
            return self._singletons[service_type]
        else:
            instance = service_info['implementation']()
            instance._container = self
            return instance

# 定义服务
class DatabaseService:
    """数据库服务"""
    
    def __init__(self):
        self.connection = "模拟数据库连接"
    
    def query(self, sql):
        print(f"   执行SQL: {sql}")
        return f"查询结果: {sql}"

class LoggingService:
    """日志服务"""
    
    def __init__(self):
        self.logs = []
    
    def log(self, message):
        self.logs.append(message)
        print(f"   日志: {message}")

class CacheService:
    """缓存服务"""
    
    def __init__(self):
        self.cache = {}
    
    def get(self, key):
        return self.cache.get(key)
    
    def set(self, key, value):
        self.cache[key] = value
        print(f"   缓存设置: {key} = {value}")

# 使用依赖注入的服务类
class UserService(metaclass=DIMeta):
    """用户服务"""
    
    @inject(DatabaseService)
    db_service = None
    
    @inject(LoggingService)
    logger = None
    
    def get_user(self, user_id):
        self.logger.log(f"获取用户 {user_id}")
        return self.db_service.query(f"SELECT * FROM users WHERE id = {user_id}")

class ProductService(metaclass=DIMeta):
    """产品服务"""
    
    @inject(DatabaseService)
    db_service = None
    
    @inject(CacheService)
    cache = None
    
    def get_product(self, product_id):
        # 先从缓存获取
        cached = self.cache.get(f"product_{product_id}")
        if cached:
            return cached
        
        # 从数据库获取
        result = self.db_service.query(f"SELECT * FROM products WHERE id = {product_id}")
        self.cache.set(f"product_{product_id}", result)
        return result

# 测试依赖注入框架
print(f"   依赖注入框架测试:")

# 创建容器
container = DIContainer()

# 注册服务
container.register(DatabaseService, singleton=True)
container.register(LoggingService, singleton=True)
container.register(CacheService)

# 创建服务实例
print(f"\n     创建用户服务:")
user_service = container.get(UserService)
user_result = user_service.get_user(123)
print(f"       {user_result}")

print(f"\n     创建产品服务:")
product_service = container.get(ProductService)
product_result1 = product_service.get_product(456)
print(f"       {product_result1}")

# 测试缓存
print(f"\n     测试缓存:")
product_result2 = product_service.get_product(456)  # 应该从缓存获取
print(f"       {product_result2}")

# 2. API框架元类
print(f"\n📝 2. API框架元类:")

class APIMeta(type):
    """API框架元类"""
    
    def __new__(mcs, name, bases, namespace):
        # 收集路由定义
        routes = {}
        for key, value in namespace.items():
            if hasattr(value, '_route'):
                route_info = value._route
                routes[route_info['path']] = {
                    'method': route_info['method'],
                    'handler': value,
                    'name': key
                }
        
        # 添加路由信息
        namespace['_routes'] = routes
        
        # 创建路由方法
        def add_route(cls, path, method, handler):
            routes[path] = {
                'method': method,
                'handler': handler,
                'name': handler.__name__
            }
        
        def get_routes(cls):
            return routes.copy()
        
        def handle_request(cls, path, method, **kwargs):
            if path in routes:
                route = routes[path]
                if route['method'].upper() == method.upper():
                    return route['handler'](cls(), **kwargs)
                else:
                    return f"方法 {method} 不允许，只允许 {route['method']}"
            else:
                return f"路由 {path} 未找到"
        
        namespace['add_route'] = classmethod(add_route)
        namespace['get_routes'] = classmethod(get_routes)
        namespace['handle_request'] = classmethod(handle_request)
        
        return super().__new__(mcs, name, bases, namespace)

def route(path, method='GET'):
    """路由装饰器"""
    def decorator(func):
        func._route = {'path': path, 'method': method}
        return func
    return decorator

class APIBase(metaclass=APIMeta):
    """API基类"""
    pass

class UserAPI(APIBase):
    """用户API"""
    
    @route('/users', 'GET')
    def get_users(self):
        return ["用户1", "用户2", "用户3"]
    
    @route('/users/{id}', 'GET')
    def get_user(self, id):
        return f"获取用户 {id} 的信息"
    
    @route('/users', 'POST')
    def create_user(self, name, email):
        return f"创建用户: {name} ({email})"
    
    @route('/users/{id}', 'PUT')
    def update_user(self, id, name=None, email=None):
        return f"更新用户 {id}: name={name}, email={email}"
    
    @route('/users/{id}', 'DELETE')
    def delete_user(self, id):
        return f"删除用户 {id}"

class ProductAPI(APIBase):
    """产品API"""
    
    @route('/products', 'GET')
    def get_products(self):
        return ["产品1", "产品2", "产品3"]
    
    @route('/products/{id}', 'GET')
    def get_product(self, id):
        return f"获取产品 {id} 的信息"

# 测试API框架
print(f"\n   API框架测试:")

print(f"\n     UserAPI 路由:")
user_routes = UserAPI.get_routes()
for path, route_info in user_routes.items():
    print(f"       {path} [{route_info['method']}] -> {route_info['name']}")

print(f"\n     测试API请求:")

# GET请求
result1 = UserAPI.handle_request('/users', 'GET')
print(f"       GET /users: {result1}")

result2 = UserAPI.handle_request('/users/123', 'GET')
print(f"       GET /users/123: {result2}")

# POST请求
result3 = UserAPI.handle_request('/users', 'POST', name="张三", email="zhangsan@example.com")
print(f"       POST /users: {result3}")

# PUT请求
result4 = UserAPI.handle_request('/users/123', 'PUT', name="李四")
print(f"       PUT /users/123: {result4}")

# DELETE请求
result5 = UserAPI.handle_request('/users/123', 'DELETE')
print(f"       DELETE /users/123: {result5}")

# 错误请求
result6 = UserAPI.handle_request('/invalid', 'GET')
print(f"       GET /invalid: {result6}")

result7 = UserAPI.handle_request('/users', 'POST')  # 缺少参数
print(f"       POST /users (无参数): {result7}")

# 3. 配置管理框架
print(f"\n📝 3. 配置管理框架:")

class ConfigMeta(type):
    """配置管理元类"""
    
    def __new__(mcs, name, bases, namespace):
        # 收集配置字段
        config_fields = {}
        for key, value in namespace.items():
            if isinstance(value, ConfigField):
                config_fields[key] = value
                del namespace[key]  # 从命名空间中移除
        
        # 添加配置字段信息
        namespace['_config_fields'] = config_fields
        
        # 创建加载配置方法
        def load_config(cls, config_dict):
            """加载配置"""
            loaded = {}
            for field_name, field in config_fields.items():
                if field_name in config_dict:
                    value = config_dict[field_name]
                    try:
                        validated_value = field.validate(value)
                        setattr(cls, field_name, validated_value)
                        loaded[field_name] = validated_value
                    except Exception as e:
                        print(f"   配置字段 {field_name} 验证失败: {e}")
                elif field.default is not None:
                    setattr(cls, field_name, field.default)
                    loaded[field_name] = field.default
                elif field.required:
                    raise ValueError(f"必需的配置字段 {field_name} 未提供")
            
            print(f"   配置加载完成，加载了 {len(loaded)} 个字段")
            return loaded
        
        # 创建获取配置方法
        def get_config(cls):
            """获取当前配置"""
            config = {}
            for field_name in config_fields:
                config[field_name] = getattr(cls, field_name, None)
            return config
        
        # 创建验证配置方法
        def validate_config(cls):
            """验证配置"""
            errors = []
            for field_name, field in config_fields.items():
                value = getattr(cls, field_name, None)
                try:
                    if value is not None:
                        field.validate(value)
                except Exception as e:
                    errors.append(f"字段 {field_name}: {e}")
            return errors
        
        namespace['load_config'] = classmethod(load_config)
        namespace['get_config'] = classmethod(get_config)
        namespace['validate_config'] = classmethod(validate_config)
        
        return super().__new__(mcs, name, bases, namespace)

class ConfigField:
    """配置字段"""
    
    def __init__(self, field_type=str, default=None, required=True, validator=None):
        self.field_type = field_type
        self.default = default
        self.required = required
        self.validator = validator
    
    def validate(self, value):
        """验证值"""
        if not isinstance(value, self.field_type):
            raise TypeError(f"值应该是 {self.field_type.__name__} 类型")
        
        if self.validator:
            return self.validator(value)
        
        return value

# 配置字段创建函数
def StringConfig(default=None, required=True, validator=None):
    return ConfigField(str, default, required, validator)

def IntConfig(default=None, required=True, validator=None):
    return ConfigField(int, default, required, validator)

def BoolConfig(default=False, required=True):
    return ConfigField(bool, default, required)

def FloatConfig(default=None, required=True, validator=None):
    return ConfigField(float, default, required, validator)

# 验证函数
def validate_port(port):
    if not (1 <= port <= 65535):
        raise ValueError("端口号必须在 1-65535 之间")
    return port

def validate_email(email):
    if '@' not in email:
        raise ValueError("邮箱格式不正确")
    return email

class DatabaseConfig(metaclass=ConfigMeta):
    """数据库配置"""
    host = StringConfig(default="localhost")
    port = IntConfig(default=5432, validator=validate_port)
    database = StringConfig(required=True)
    username = StringConfig(required=True)
    password = StringConfig(required=True)
    ssl_enabled = BoolConfig(default=False)

class AppConfig(metaclass=ConfigMeta):
    """应用配置"""
    app_name = StringConfig(required=True)
    debug = BoolConfig(default=False)
    log_level = StringConfig(default="INFO", validator=lambda x: x in ['DEBUG', 'INFO', 'WARNING', 'ERROR'])
    max_workers = IntConfig(default=4, validator=lambda x: x > 0)
    timeout = FloatConfig(default=30.0, validator=lambda x: x > 0)
    admin_email = StringConfig(validator=validate_email)

# 测试配置管理框架
print(f"\n   配置管理框架测试:")

# 数据库配置
db_config_dict = {
    'host': 'localhost',
    'port': 5432,
    'database': 'myapp',
    'username': 'admin',
    'password': 'secret123',
    'ssl_enabled': True
}

print(f"\n     加载数据库配置:")
DatabaseConfig.load_config(db_config_dict)
print(f"       当前配置: {DatabaseConfig.get_config()}")

# 应用配置
app_config_dict = {
    'app_name': 'MyApp',
    'debug': True,
    'log_level': 'DEBUG',
    'max_workers': 8,
    'timeout': 60.0,
    'admin_email': 'admin@example.com'
}

print(f"\n     加载应用配置:")
AppConfig.load_config(app_config_dict)
print(f"       当前配置: {AppConfig.get_config()}")

# 测试配置验证
print(f"\n     测试配置验证:")
errors = AppConfig.validate_config()
if errors:
    print(f"       配置错误: {errors}")
else:
    print(f"       配置验证通过")

# 测试无效配置
print(f"\n     测试无效配置:")
invalid_config = {
    'app_name': 'MyApp',
    'max_workers': -1,  # 无效值
    'log_level': 'INVALID'  # 无效值
}

try:
    AppConfig.load_config(invalid_config)
except Exception as e:
    print(f"       配置加载失败: {e}")

print(f"\n✅ 实际框架案例完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握元类在实际框架中的应用")
print(f"   ✓ 理解元类驱动的架构设计模式")
print(f"   ✓ 能设计基于元类的框架组件")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**3.8 元类编程 [⭐⭐进阶]**
- ✅ 掌握元类的基本概念和工作原理
- ✅ 理解元类的创建方法和应用场景
- ✅ 能够运用元类解决复杂的框架设计问题
- ✅ 掌握元类的高级应用技巧
- ✅ 理解元类在框架设计中的作用
- ✅ 能运用元类解决复杂的架构问题
- ✅ 能独立设计和实现元类

### 🎯 与LangChain学习的关联

**元类编程重要性**：
- 元类是LangChain框架架构的核心技术
- LangChain使用元类实现组件的自动注册和发现
- 元类模式影响LangChain的扩展性和配置管理
- 元类为LangChain的动态组件创建提供支持

**实际应用场景**：
- LangChain的LLM组件注册使用元类模式
- Chain组件的动态构建使用元类技术
- 工具类的自动发现使用注册元类
- 配置管理系统使用配置元类

### 📚 进阶学习建议

1. **练习建议**：
   - 多在实际框架中应用元类编程
   - 深入理解元类的性能影响和优化
   - 掌握元类与装饰器的组合使用

2. **扩展学习**：
   - 学习描述符协议与元类的结合
   - 了解Python的抽象语法树(AST)操作
   - 探索动态代码生成和执行

3. **实际应用**：
   - 设计自己的框架架构
   - 实现插件系统和组件注册
   - 构建动态配置管理系统

### 🔧 常见错误与注意事项

1. **过度使用元类**：
   ```python
   # 错误：简单问题使用复杂元类
   class SimpleClass(metaclass=ComplexMeta):
       def __init__(self, value):
           self.value = value
   
   # 正确：使用普通类
   class SimpleClass:
       def __init__(self, value):
           self.value = value
   ```

2. **元类继承问题**：
   ```python
   # 错误：元类继承不正确
   class ParentClass:
       pass
   
   class ChildClass(ParentClass, metaclass=CustomMeta):
       pass  # 可能导致元类冲突
   
   # 正确：确保元类兼容性
   class ParentClass(metaclass=CustomMeta):
       pass
   
   class ChildClass(ParentClass):
       pass
   ```

3. **性能问题**：
   ```python
   # 错误：在性能敏感场景过度使用元类
   class PerformanceCritical(metaclass=HeavyMeta):
       def process(self, data):
           # 元类开销可能影响性能
           return self._transform(data)
   
   # 正确：在性能关键场景简化设计
   class PerformanceCritical:
       def process(self, data):
           return self._transform(data)
   ```

4. **命名空间污染**：
   ```python
   # 错误：元类修改命名空间不当
   class BadMeta(type):
       def __new__(mcs, name, bases, namespace):
           # 添加过多内部属性
           namespace['_internal_1'] = 'value1'
           namespace['_internal_2'] = 'value2'
           namespace['_internal_3'] = 'value3'
           return super().__new__(mcs, name, bases, namespace)
   
   # 正确：合理管理命名空间
   class GoodMeta(type):
       def __new__(mcs, name, bases, namespace):
           # 只添加必要的属性
           if '_meta_info' not in namespace:
               namespace['_meta_info'] = {}
           return super().__new__(mcs, name, bases, namespace)
   ```

5. **循环依赖**：
   ```python
   # 错误：元类循环依赖
   class MetaA(type):
       def __new__(mcs, name, bases, namespace):
           if name == 'ClassB':
               return ClassB  # 循环依赖
           return super().__new__(mcs, name, bases, namespace)
   
   # 正确：避免循环依赖
   class MetaA(type):
       def __new__(mcs, name, bases, namespace):
           # 延迟处理或使用注册模式
           return super().__new__(mcs, name, bases, namespace)
   ```

---

**🎉 恭喜完成元类编程学习！**

你已经掌握了Python元类编程的核心概念和高级应用，能够灵活运用元类实现框架级的功能定制，为后续学习面向对象设计原则以及LangChain的高级框架设计奠定了坚实基础。

## 🏆 下一步学习预告

**完成第三节：面向对象编程**：
- 3.9 面向对象设计原则 [⭐⭐进阶]

**继续后续章节**：
- 第四节：常用标准库 (9个笔记本)
- 第五节：数据处理 (9个笔记本)
- 第六节：异步编程 (9个笔记本)

继续加油，完成Python面向对象编程的最后部分！